In [1]:
import pandas as pd
import numpy as np
from dataloading import DataLoader
from eda_py import EDA
from eALS_adaptor import eALSAdaptor
from implicit.evaluation import train_test_split, ranking_metrics_at_k
from cv_py import CrossValidation
from visualize_py import Visualize

%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [2]:
dl = DataLoader()

In [3]:
user_item_co = dl.import_data('AGCO', 'CO', 'df')
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [4]:
user_item_filtered = dl.remove_low_interact_items(user_item_co, 1)
user_item_filtered_t = dl.remove_low_interact_items(user_item_co_t, 1)

In [5]:
user_item_filtered_log = dl.log_scale_df(user_item_filtered, 0.01)
user_item_filtered_log_t = dl.log_scale_df(user_item_filtered_t, 0.01)

In [6]:
user_item_csr = dl.to_csr(user_item_filtered_log)
user_item_csr_t = dl.to_csr(user_item_filtered_log_t)

In [7]:
# initial split
train, test = train_test_split(user_item_csr, 0.8, 22)
train_t, test_t = train_test_split(user_item_csr_t, 0.8, 22)

In [8]:
cv = CrossValidation(train, 5)
cv_t = CrossValidation(train_t, 5)

In [9]:
validate, calibrate = cv.split_k_fold(22)
validate_t, calibrate_t = cv_t.split_k_fold(22)

In [61]:
space_iALS = {'factors' : [64], 'regularization' : [30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 140, 160], 'alpha' : [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 'iterations' : [15]}
space_iALS_t = {'factors' : [64], 'regularization' : [30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 140, 160], 'alpha' : [0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6], 'iterations' : [15]}
hyper_ials = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_iALS, model_class='iALS')
hyper_ials_t = cv_t.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_iALS_t, model_class='iALS')

In [10]:
space_eALS_t = {'factors' : [64], 'regularization' : [5, 10, 20, 30, 40, 50, 60, 70, 80], 'alpha' : [0.001, 0.01, 0.1, 1, 2], 
'w0' : [50, 100, 500, 1000, 5000, 10000], 'iterations' : [15]}
hyper_eals_t = cv_t.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_eALS_t, model_class='eALS')

converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items t

In [98]:
space_BPR = {'factors' : [64], 'regularization' : [0.005, 0.01, 0.03, 0.05, 0.07, 0.1, 0.12], 'learning_rate' : [0.001, 0.003, 0.005, 0.007, 0.01, 0.015, 0.02, 0.03, 0.05], 'iterations' : [15]}
hyper_bpr_add = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_BPR, model_class='BPR')

In [170]:
space_LMF = {'factors' : [64], 'regularization' : [, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'learning_rate' : [0.5, 0.7, 1.0, 1.2, 1.5, 1.7, 2.0], 
'iterations' : [15], 'neg_prop': [0, 30, 40, 50, 60, 70, 80]}
hyper_lmf_add = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_LMF, model_class='LMF')